In [1]:
import cv2
from keras.models import load_model
import numpy as np
import os

model = load_model('emojinator.h5')

def get_emojis():
    emojis_folder = 'C:/Users/romit/Eckovation ML/Emojinator/Hand_emo/'
    emojis = []
    for emoji in range(len(os.listdir(emojis_folder))):
        print(emoji)
        emojis.append(cv2.imread(emojis_folder+str(emoji)+'.png', -1))
    return emojis

def keras_predict(model, image):
    processed = keras_process_image(image)
    pred_probab = model.predict(processed)[0]
    pred_class = list(pred_probab).index(max(pred_probab))
    return max(pred_probab), pred_class

def keras_process_image(img):
    image_x = 50
    image_y = 50
    img = cv2.resize(img, (image_x, image_y))
    img = np.array(img, dtype=np.float32)
    img = np.reshape(img, (-1, image_x, image_y, 1))
    return img

def overlay(image, emoji, x,y,w,h):
    emoji = cv2.resize(emoji, (w, h))
    try:
        image[y:y+h, x:x+w] = blend_transparent(image[y:y+h, x:x+w], emoji)
    except:
        pass
    return image

def blend_transparent(face_img, overlay_t_img):
    # Split out the transparency mask from the colour info
    overlay_img = overlay_t_img[:,:,:3] # Grab the BRG planes
    overlay_mask = overlay_t_img[:,:,3:]  # And the alpha plane

    # Again calculate the inverse mask
    background_mask = 255 - overlay_mask

    # Turn the masks into three channel, so we can use them as weights
    overlay_mask = cv2.cvtColor(overlay_mask, cv2.COLOR_GRAY2BGR)
    background_mask = cv2.cvtColor(background_mask, cv2.COLOR_GRAY2BGR)

    # Create a masked out face image, and masked out overlay
    # We convert the images to floating point in range 0.0 - 1.0
    face_part = (face_img * (1 / 255.0)) * (background_mask * (1 / 255.0))
    overlay_part = (overlay_img * (1 / 255.0)) * (overlay_mask * (1 / 255.0))

    # And finally just add them together, and rescale it back to an 8bit integer image
    return np.uint8(cv2.addWeighted(face_part, 255.0, overlay_part, 255.0, 0.0))

emojis = get_emojis()
cap = cv2.VideoCapture(0)
x, y, w, h = 300, 50, 350, 350

while (cap.isOpened()):
    ret, img = cap.read()
    img = cv2.flip(img, 1)
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    mask2 = cv2.inRange(hsv, np.array([2, 50, 60]), np.array([25, 150, 255]))
    res = cv2.bitwise_and(img, img, mask=mask2)
    gray = cv2.cvtColor(res, cv2.COLOR_BGR2GRAY)
    median = cv2.GaussianBlur(gray, (5, 5), 0)

    kernel_square = np.ones((5, 5), np.uint8)
    dilation = cv2.dilate(median, kernel_square, iterations=2)
    opening = cv2.morphologyEx(dilation, cv2.MORPH_CLOSE, kernel_square)
    ret, thresh = cv2.threshold(opening, 30, 255, cv2.THRESH_BINARY)

    thresh = thresh[y:y + h, x:x + w]
    contours = cv2.findContours(thresh.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)[0]
    if len(contours) > 0:
        contour = max(contours, key=cv2.contourArea)
        if cv2.contourArea(contour) > 2500:
            x, y, w1, h1 = cv2.boundingRect(contour)
            newImage = thresh[y:y + h1, x:x + w1]
            newImage = cv2.resize(newImage, (50, 50))
            pred_probab, pred_class = keras_predict(model, newImage)
            print(pred_class, pred_probab)
            img = overlay(img, emojis[pred_class], 400, 250, 90, 90)

    x, y, w, h = 300, 50, 350, 350
    cv2.imshow("Frame", img)
    cv2.imshow("Contours", thresh)
    k = cv2.waitKey(10)
    if k == 27:
        break

0
1
2
3
4
5
6
7
8
9
10
11
12
3 0.88617724
9 0.7024626
3 0.61758167
9 0.94027036
9 0.99999964
9 0.9999881
9 1.0
9 0.999998
9 1.0
9 0.99999964
9 0.9999999
9 0.99995863
9 0.99999404
9 0.9999994
9 0.9999999
9 0.57988054
9 0.99997973
9 0.9999976
9 0.9999939
9 1.0
9 0.9999989
9 0.9999937
9 0.99999845
9 0.9999993
9 1.0
9 0.99999857
9 0.9999999
9 0.99999416
9 0.99999976
9 0.9999999
9 0.99999833
9 0.99999905
9 0.9999988
9 0.9999988
9 0.99999714
9 0.9999261
9 0.9994741
9 0.99972636
9 0.99996686
9 0.9999871
9 0.99999833
9 0.99997544
9 0.9999714
9 0.9999852
9 0.9999932
9 0.999997
9 0.99999785
9 0.99173224
9 0.99999917
9 0.9999958
9 0.99999595
9 0.999995
9 0.99999666
9 0.99999976
9 0.99996996
9 0.9999931
9 0.9999709
9 0.9999883
9 0.9999876
9 0.99997497
9 0.9998324
9 0.99999785
9 0.9999825
9 0.99998736
9 0.9998091
9 0.9998927
9 0.99999297
9 0.99999225
9 0.9999038
9 0.9999746
9 0.9995459
9 0.9999894
9 0.9998361
9 0.9999957
9 0.9997023
9 0.9997023
9 0.9999342
9 0.9999571
9 0.9999571
9 0.9999956
9 0.99

7 0.99984884
7 0.998686
7 0.99905545
7 0.99784255
7 0.9954253
7 0.9945791
7 0.9999988
7 0.99723864
7 0.99779797
7 0.99937433
7 0.99957865
7 0.99957865
7 0.99964905
7 0.99958485
4 0.5066792
4 0.5066792
8 0.5024172
1 0.9999933
8 0.47251612
8 0.47251612
5 0.95841503
1 0.59810036
1 0.6787426
1 0.6787426
1 0.9079524
1 0.7703058
8 0.8202674
8 0.8202674
2 0.65724283
8 0.60587084
8 0.51285774
8 0.51285774
8 0.7026271
8 0.59207374
8 0.7912476
8 0.7912476
8 0.81033105
8 0.88904417
8 0.723564
8 0.723564
8 0.56112057
8 0.58003825
8 0.56614065
8 0.56614065
1 0.651109
1 0.42455932
1 0.42455932
1 0.4418037
1 0.73345
8 0.86388105
8 0.86388105
2 0.7808533
11 0.7935702
11 0.7935702
8 0.99960893
8 0.99999607
8 0.9998129
8 0.9998129
8 0.54386777
5 0.9921559
5 0.9964604
5 0.9964604
5 0.9934255
5 0.89386356
5 0.89386356
5 0.99834216
5 0.99976057
5 0.9960348
5 0.9960348
2 0.46183494
5 0.64868987
5 0.99981326
5 0.99791294
5 0.99791294
5 0.94911677
5 0.8737753
5 0.823764
5 0.823764
5 0.9998202
5 0.43795913
5 0

9 1.0
9 1.0
9 1.0
9 1.0
9 1.0
9 1.0
9 1.0
9 1.0
9 1.0
9 0.9999999
9 1.0
9 1.0
9 1.0
9 1.0
9 1.0
9 1.0
9 1.0
9 1.0
9 0.99999976
9 0.99999976
9 1.0
9 1.0
9 1.0
9 1.0
9 1.0
9 1.0
9 1.0
9 1.0
9 1.0
9 1.0
9 1.0
9 1.0
9 1.0
9 1.0
9 1.0
9 1.0
9 1.0
9 1.0
9 1.0
9 1.0
9 0.99951494
5 0.7127396
9 0.9949103
9 0.9949103
1 0.9015289
8 0.99929404
3 0.743204
3 0.743204
3 0.8418436
1 0.9975285
1 0.9980355
1 0.9980355
1 0.99819905
1 0.9976901
1 0.99271506
1 0.99271506
1 0.99870074
1 0.9947129
1 0.99798906
1 0.99798906
1 0.9970009
1 0.9992347
2 0.59501857
2 0.59501857
9 0.7037867
10 0.9966023
5 0.972325
5 0.972325
9 0.5234092
9 1.0
9 1.0
9 0.9999999
9 1.0
9 1.0
9 1.0
9 1.0
9 1.0
9 1.0
9 1.0
9 1.0
9 1.0
9 1.0
9 1.0
9 1.0
9 1.0
9 1.0
9 1.0
9 1.0
9 1.0
9 1.0
9 1.0
9 1.0
9 1.0
9 1.0
9 1.0
9 1.0
9 1.0
9 1.0
9 1.0
9 1.0
9 1.0
9 0.9999999
9 1.0
9 1.0
9 1.0
9 1.0
9 0.9999999
9 0.9999999
9 0.99999106
9 0.9886335
5 0.90726995
9 0.99560463
9 0.8926957
9 0.8926957
1 0.99935955
1 0.9253031
1 0.99743265
1 0.99743265
1

9 0.9999999
9 0.99999964
9 0.99999785
9 0.99999785
9 0.9999993
9 1.0
9 1.0
9 0.9999999
9 0.9999999
9 0.99999976
9 0.99999964
9 0.99999964
9 1.0
9 0.9999964
9 0.9999964
9 1.0
9 0.9999999
9 0.9999995
9 0.9999995
9 0.99999845
9 0.9999982
9 0.9999931
9 0.9999931
9 0.99982387
9 0.9999999
9 0.99999726
9 0.99999726
9 0.9999534
9 0.9999994
9 0.99999785
9 0.99999785
9 0.99997723
9 0.9999993
9 0.99999964
9 0.9999664
9 0.9999664
9 0.99937767
9 0.999936
9 0.999936
9 0.99998796
9 0.99999475
9 0.998569
9 0.998569
9 0.9985598
9 0.93766445
9 0.9988086
9 0.9988086
9 0.99982786
9 0.99443936
9 0.99930704
9 0.99930704
9 0.99998045
9 0.99997926
9 0.99999714
9 0.99999714
9 0.9999993
9 0.99946994
9 0.9395073
9 0.9395073
9 0.99777824
9 0.99985015
9 0.9999982
9 0.9999982
9 0.99996984
9 0.99973494
9 0.99989843
9 0.99989843
9 0.98228663
9 0.99999964
9 0.9999677
9 0.9999677
9 0.99989533
9 0.99998
9 0.9995741
9 0.9995741
9 0.9999982
9 0.99990237
9 0.99999213
9 0.99999213
9 0.9999714
9 0.9998956
9 0.9998956
9 0.999

9 0.96038187
9 0.96038187
9 0.9849917
9 0.9999746
9 0.9999746
9 0.9875187
9 0.9744221
9 0.9932562
9 0.9932562
9 0.9920798
9 0.998042
9 0.99036497
8 0.5112484
8 0.5112484
9 0.998926
9 0.9990392
8 0.7817384
8 0.7817384
9 0.9998838
9 0.95759326
9 0.95759326
9 0.51003474
9 0.98491085
9 0.9999596
9 0.9999596
9 0.9995666
9 0.99997175
9 0.99997175
9 0.9997029
9 0.9999963
9 0.99986863
9 0.99999774
9 0.99999774
9 0.9999999
9 1.0
9 1.0
9 0.9999999
9 0.99999774
9 0.99999976
9 1.0
9 1.0
9 1.0
9 0.9999999
9 0.9999999
9 1.0
9 1.0
9 1.0
9 1.0
9 1.0
9 0.99999976
9 0.9999999
9 0.9999999
9 1.0
9 1.0
9 1.0
9 1.0
9 1.0
9 1.0
9 1.0
9 1.0
9 1.0
9 1.0
9 1.0
9 0.99999976
9 0.99999976
9 0.9999999
9 0.99999917
9 1.0
9 1.0
9 1.0
9 1.0
9 0.9999999
9 0.9999999
9 1.0
9 1.0
9 1.0
9 1.0
9 0.9999999
9 1.0
9 1.0
9 0.9999999
9 1.0
9 1.0
9 1.0
9 0.99999976
9 1.0
9 1.0
9 0.99999964
9 0.9999987
9 0.9999999
9 0.9999999
9 0.9999999
9 0.99999964
9 0.99999964
9 0.9999932
9 1.0
9 0.9999999
9 0.9999999
9 0.9999999
9 0.9999974
9 

9 0.9953145
9 0.9953145
9 0.999997
9 0.9999188
9 0.99999964
9 0.99999964
9 0.98750055
9 0.9997209
9 0.99963415
9 0.99963415
9 0.9981123
9 0.9095816
9 0.9095816
9 0.956617
9 0.78418636
9 0.9970561
9 0.9970561
9 0.99999356
9 0.99999857
9 0.9987036
9 0.9987036
9 0.99877495
9 0.99999905
9 0.99999905
9 0.9999685
9 0.99995947
9 0.9983883
9 0.99740344
9 0.99999464
9 0.99999464
9 0.99953973
9 0.8693836
9 0.8693836
9 0.99956137
9 0.9999931
9 0.9997645
9 0.9997645
9 0.9844801
8 0.52924305
8 0.52924305
9 0.99513996
9 0.99837095
9 0.99963677
9 0.99963677
9 0.98509884
9 0.99824786
9 0.99999523
9 0.99999523
9 0.9955336
9 0.99773586
9 0.84711486
9 0.84711486
9 0.99568295
9 0.99838245
9 0.981339
9 0.981339
9 0.9999026
9 0.99995935
9 0.98787653
9 0.98787653
9 0.99283993
9 0.99974245
9 0.9953621
9 0.9953621
9 0.9994592
9 0.9993262
9 0.9788327
9 0.9788327
9 0.99648917
9 0.9999964
9 0.91396767
9 0.91396767
9 0.9999769
9 0.99992037
9 0.9994831
9 0.9994831
9 0.9979468
9 0.99944144
9 0.99718106
9 0.99718106


9 0.9999895
9 0.99996924
9 0.9961557
9 0.9980539
9 0.9980539
9 0.99870896
9 0.990621
9 0.9907855
9 0.9907855
9 0.9952583
9 0.99748373
9 0.9999391
9 0.9999391
9 0.9997855
9 0.99966025
9 0.99996626
9 0.99996626
9 0.9999976
9 0.99998474
9 0.99998474
9 0.9999987
9 0.9999881
9 0.9999864
9 0.9999864
9 0.99999297
9 0.99999976
9 0.9999995
9 0.9999999
9 0.9999999
9 0.9999999
9 0.9999999
9 0.9999999
9 0.99999976
9 0.99999905
9 0.9999999
9 0.9999999
9 0.99999905
9 0.9999999
9 0.9999994
9 0.9999994
9 0.9999999
9 0.9999995
9 0.9999995
9 0.99999976
9 0.9999994
9 0.9999999
9 0.9999999
9 0.99999976
9 0.9999999
9 0.9999994
9 0.9999994
9 0.9999999
9 1.0
9 0.99999857
9 0.99999857
9 0.9999994
9 1.0
9 1.0
9 0.99999917
9 1.0
9 0.9999994
9 0.9999994
9 0.99999976
9 0.99999905
9 0.99999905
9 0.9999994
9 0.99999976
9 0.99999976
9 0.99999976
9 0.99999976
9 0.99999964
9 0.9999999
9 0.9999999
9 0.9999999
9 0.9999999
9 0.99999976
9 0.99999976
9 0.99999964
9 0.9999999
9 0.9999999
9 0.9999994
9 0.99999905
9 0.9999997

1 0.9997465
1 0.99053824
1 0.945626
5 0.51879114
5 0.87996805
1 0.98276204
1 0.9541072
1 0.99687874
1 0.8127181
5 0.9837654
1 0.95488876
1 0.9289721
1 0.9270281
1 0.8953423
1 0.96129143
1 0.9752686
1 0.98192227
1 0.99567115
5 0.83078575
5 0.6282936
1 0.9981528
1 0.99696344
1 0.7078649
1 0.99135625
1 0.766556
5 0.84308726
1 0.9390292
1 0.9190281
1 0.9823375
1 0.86926526
1 0.9892525
1 0.99290234
1 0.81583357
1 0.630698
5 0.55926013
5 0.60350406
11 0.75327843
11 0.6595856
11 0.669467
11 0.461823
11 0.7163707
11 0.6449707
11 0.48882872
5 0.8610363
11 0.47058508
5 0.6554472
11 0.7242586
5 0.65289223
11 0.44683987
11 0.57230115
5 0.59230393
5 0.6959664
5 0.83659655
5 0.6845608
5 0.56214494
5 0.8255763
5 0.80376494
4 0.98079896
4 0.9412003
4 0.92719537
4 0.68637806
4 0.96360815
4 0.74796396
4 0.84949374
4 0.9125815
4 0.5518741
4 0.95566845
4 0.7316098
4 0.98064053
4 0.87450886
4 0.71265864
10 0.8151787
4 0.61190736
4 0.5222987
8 0.9216237
3 0.9534426
3 0.88835955
1 0.81082034
5 0.9984409
2 0.

7 0.99863464
7 0.9972639
4 0.51606244
7 0.99999523
7 0.85372293
4 0.92241156
4 0.99074495
4 0.87661356
4 0.9994153
4 0.9988053
4 0.9991609
6 0.6317918
1 0.99441755
1 0.9999926
1 0.99996126
1 0.9999943
1 0.9998872
1 0.99999166
1 0.9999989
1 0.9999988
1 0.99999774
1 0.99999726
1 0.9999969
1 0.99998844
8 0.54075384
1 0.9999964
1 0.99999475
1 0.99999547
1 0.9843763
1 0.9031077
1 0.6690639
1 0.98335373
1 0.9815973
1 0.78270483
1 0.48777577
1 0.9802551
1 0.67980486
1 0.62482345
1 0.5462685
11 0.51076186
11 0.71960634
1 0.5260498
1 0.73097545
11 0.6008944
1 0.5335408
11 0.8023421
1 0.5653939
11 0.8037287
11 0.83142704
11 0.93384403
1 0.5077626
1 0.5548085
11 0.5920269
11 0.4277176
1 0.48444518
11 0.46240228
11 0.7948272
11 0.9469509
11 0.91690654
11 0.9838234
11 0.9884272
11 0.99670714
11 0.92895156
11 0.9984555
4 0.7794786
4 0.93831486
11 0.96903753
11 0.98410296
11 0.99894816
11 0.9866555
11 0.99950767
11 0.96732795
11 0.99822134
11 0.9912186
11 0.9975248
11 0.9994887
11 0.99963534
7 0.9781